In [1]:
import torch

In [2]:
inputs=torch.tensor([[0.43,0.15,0.89],
                    [0.55,0.87,0.66],
                    [0.57,0.85,0.64],
                    [0.22,0.58,0.33],
                    [0.77,0.25,0.10],
                    [0.05,0.80,0.55]]
                   )

In [3]:
x2=inputs[1]
din=inputs.shape[1]
dout=2

In [4]:
torch.manual_seed(123)
wquery=torch.nn.Parameter(torch.rand(din,dout),requires_grad=False)
wkey=torch.nn.Parameter(torch.rand(din,dout),requires_grad=False)
wvalue=torch.nn.Parameter(torch.rand(din,dout),requires_grad=False)

In [5]:
print(wquery)

Parameter containing:
tensor([[0.2961, 0.5166],
        [0.2517, 0.6886],
        [0.0740, 0.8665]])


In [6]:
print(wkey)

Parameter containing:
tensor([[0.1366, 0.1025],
        [0.1841, 0.7264],
        [0.3153, 0.6871]])


In [7]:
print(wvalue)

Parameter containing:
tensor([[0.0756, 0.1966],
        [0.3164, 0.4017],
        [0.1186, 0.8274]])


In [8]:
query2=x2 @ wquery
key2= x2 @ wkey
value2= x2 @ wvalue
print(query2)

tensor([0.4306, 1.4551])


In [9]:
keys = inputs @ wkey
values = inputs @ wvalue
queries = inputs @ wquery

print("keys.shape:",keys.shape)
print("values.shape:",values.shape)
print("queries.shape:",queries.shape)

keys.shape: torch.Size([6, 2])
values.shape: torch.Size([6, 2])
queries.shape: torch.Size([6, 2])


In [10]:
keys2=keys[1]
attnscore22=query2.dot(key2)
print(attnscore22)

tensor(1.8524)


In [11]:
attnscore2=query2 @ keys.T
print(attnscore2)

tensor([1.2705, 1.8524, 1.8111, 1.0795, 0.5577, 1.5440])


In [12]:
attn_scores = queries @ keys.T
print(attn_scores)

tensor([[0.9231, 1.3545, 1.3241, 0.7910, 0.4032, 1.1330],
        [1.2705, 1.8524, 1.8111, 1.0795, 0.5577, 1.5440],
        [1.2544, 1.8284, 1.7877, 1.0654, 0.5508, 1.5238],
        [0.6973, 1.0167, 0.9941, 0.5925, 0.3061, 0.8475],
        [0.6114, 0.8819, 0.8626, 0.5121, 0.2707, 0.7307],
        [0.8995, 1.3165, 1.2871, 0.7682, 0.3937, 1.0996]])


In [13]:
dk=keys.shape[-1]
attnweights2=torch.softmax(attnscore2/dk**0.5, dim = -1)
print(attnweights2)
print(dk)

tensor([0.1500, 0.2264, 0.2199, 0.1311, 0.0906, 0.1820])
2


In [14]:
contextvec2=attnweights2 @ values
print(contextvec2)

tensor([0.3061, 0.8210])


In [15]:
import torch.nn as nn

In [16]:
class selfattentionv1(nn.Module):
    def __init__(self,din,dout):
        super().__init__()
        self.wquery=nn.Parameter(torch.rand(din,dout))
        self.wkey=nn.Parameter(torch.rand(din,dout))
        self.wvalue=nn.Parameter(torch.rand(din,dout))
    def forward(self,x):
        keys = x @ self.wkey
        queries= x @ self.wquery
        values = x @ self.wvalue

        attnscores= queries @ keys.T
        attnweights = torch.softmax(attnscores/keys.shape[-1]**0.5,dim=-1)

        context_vector= attnweights @ values
        return context_vector

In [17]:
torch.manual_seed(123)
sav1=selfattentionv1(din,dout)
print(sav1(inputs))

tensor([[0.2996, 0.8053],
        [0.3061, 0.8210],
        [0.3058, 0.8203],
        [0.2948, 0.7939],
        [0.2927, 0.7891],
        [0.2990, 0.8040]], grad_fn=<MmBackward0>)


In [18]:
class selfattentionv2(nn.Module):
    def __init__(self,din,dout,qkv_bias=False):
        super().__init__()
        self.wquery=nn.Linear(din,dout,bias=qkv_bias)
        self.wkey=nn.Linear(din,dout,bias=qkv_bias)
        self.wvalue=nn.Linear(din,dout,bias=qkv_bias)
    def forward(self,x):
        keys =self.wkey(x)
        queries=self.wquery(x)
        values =self.wvalue(x)

        attnscores= queries @ keys.T
        attnweights = torch.softmax(attnscores/keys.shape[-1]**0.5,dim=-1)

        context_vector= attnweights @ values
        return context_vector

In [19]:
torch.manual_seed(123)
sav2=selfattentionv2(din,dout)
print(sav2(inputs))

tensor([[-0.5337, -0.1051],
        [-0.5323, -0.1080],
        [-0.5323, -0.1079],
        [-0.5297, -0.1076],
        [-0.5311, -0.1066],
        [-0.5299, -0.1081]], grad_fn=<MmBackward0>)


In [20]:
inputs=torch.tensor([[0.43,0.15,0.89],
                    [0.55,0.87,0.66],
                    [0.57,0.85,0.64],
                    [0.22,0.58,0.33],
                    [0.77,0.25,0.10],
                    [0.05,0.80,0.55]]
                   )

In [21]:
queries = sav2.wquery(inputs)
keys = sav2.wkey(inputs)
attnscores= queries @ keys.T
attnweights = torch. softmax(attnscores/keys.shape[-1]**0.5,dim=1)
print(attnweights)

tensor([[0.1717, 0.1762, 0.1761, 0.1555, 0.1627, 0.1579],
        [0.1636, 0.1749, 0.1746, 0.1612, 0.1605, 0.1652],
        [0.1637, 0.1749, 0.1746, 0.1611, 0.1606, 0.1651],
        [0.1636, 0.1704, 0.1702, 0.1652, 0.1632, 0.1674],
        [0.1667, 0.1722, 0.1721, 0.1618, 0.1633, 0.1639],
        [0.1624, 0.1709, 0.1706, 0.1654, 0.1625, 0.1682]],
       grad_fn=<SoftmaxBackward0>)


In [22]:
contextlength=attnscores.shape[0]
masksimple=torch.tril(torch.ones(contextlength,contextlength))
print(masksimple)

tensor([[1., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1.]])


In [23]:
maskedsimple= attnweights * masksimple
print(maskedsimple)

tensor([[0.1717, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1636, 0.1749, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1637, 0.1749, 0.1746, 0.0000, 0.0000, 0.0000],
        [0.1636, 0.1704, 0.1702, 0.1652, 0.0000, 0.0000],
        [0.1667, 0.1722, 0.1721, 0.1618, 0.1633, 0.0000],
        [0.1624, 0.1709, 0.1706, 0.1654, 0.1625, 0.1682]],
       grad_fn=<MulBackward0>)


In [24]:
rowssum=maskedsimple.sum(dim=1,keepdim=True)
normmaskedsimple = maskedsimple/rowssum
print(normmaskedsimple)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.4833, 0.5167, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3190, 0.3408, 0.3402, 0.0000, 0.0000, 0.0000],
        [0.2445, 0.2545, 0.2542, 0.2468, 0.0000, 0.0000],
        [0.1994, 0.2060, 0.2058, 0.1935, 0.1953, 0.0000],
        [0.1624, 0.1709, 0.1706, 0.1654, 0.1625, 0.1682]],
       grad_fn=<DivBackward0>)


In [26]:
mask=torch.triu(torch.ones(contextlength,contextlength),diagonal=1)
print(mask)

tensor([[0., 1., 1., 1., 1., 1.],
        [0., 0., 1., 1., 1., 1.],
        [0., 0., 0., 1., 1., 1.],
        [0., 0., 0., 0., 1., 1.],
        [0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0.]])


In [27]:
masked=attnscores.masked_fill(mask.bool(),-torch.inf)
print(masked)

tensor([[0.3111,   -inf,   -inf,   -inf,   -inf,   -inf],
        [0.1655, 0.2602,   -inf,   -inf,   -inf,   -inf],
        [0.1667, 0.2602, 0.2577,   -inf,   -inf,   -inf],
        [0.0510, 0.1080, 0.1064, 0.0643,   -inf,   -inf],
        [0.1415, 0.1875, 0.1863, 0.0987, 0.1121,   -inf],
        [0.0476, 0.1192, 0.1171, 0.0731, 0.0477, 0.0966]],
       grad_fn=<MaskedFillBackward0>)


In [28]:
attnweights=torch.softmax(masked/keys.shape[-1]**0.5,dim=1)
print(attnweights)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.4833, 0.5167, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3190, 0.3408, 0.3402, 0.0000, 0.0000, 0.0000],
        [0.2445, 0.2545, 0.2542, 0.2468, 0.0000, 0.0000],
        [0.1994, 0.2060, 0.2058, 0.1935, 0.1953, 0.0000],
        [0.1624, 0.1709, 0.1706, 0.1654, 0.1625, 0.1682]],
       grad_fn=<SoftmaxBackward0>)


In [29]:
torch.manual_seed(123)
dropout=torch.nn.Dropout(0.5)
example=torch.ones(6,6)
print(dropout(example))

tensor([[2., 2., 0., 2., 2., 0.],
        [0., 0., 0., 2., 0., 2.],
        [2., 2., 2., 2., 0., 2.],
        [0., 2., 2., 0., 0., 2.],
        [0., 2., 0., 2., 0., 2.],
        [0., 2., 2., 2., 2., 0.]])


In [30]:
print(dropout(attnweights))

tensor([[2.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.6816, 0.6804, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.5085, 0.4936, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.3906, 0.0000],
        [0.3249, 0.3418, 0.0000, 0.3308, 0.3249, 0.3363]],
       grad_fn=<MulBackward0>)


In [31]:
batch=torch.stack((inputs,inputs),dim=0)
print(batch.shape)

torch.Size([2, 6, 3])


In [32]:
class CausalAttention(nn.Module):

    def __init__(self,din,dout,contextlength,dropout,qkvbias=False):
        super().__init__()
        self.dout=dout
        self.wquery=nn.Linear(din,dout,bias=qkvbias)
        self.wkey=nn.Linear(din,dout,bias=qkvbias)
        self.wvalue=nn.Linear(din,dout,bias=qkvbias)
        self.dropout=nn.Dropout(dropout)
        self.register_buffer('mask',torch.triu(torch.ones(contextlength,contextlength),diagonal=1))
        
    def forward(self,x):
        b,numtokens,din = x.shape
        keys =self.wkey(x)
        queries=self.wquery(x)
        values =self.wvalue(x)

        attnscores= queries @ keys.transpose(1,2)
        attnscores.masked_fill_(self.mask.bool()[:numtokens,:numtokens],-torch.inf)
        
        attnweights = torch.softmax(attnscores/keys.shape[-1]**0.5,dim=-1)
        attnweights = self.dropout(attnweights)

        context_vector= attnweights @ values
        return context_vector
    

In [33]:
torch.manual_seed(123)
contextlength=batch.shape[1]
ca=CausalAttention(din,dout,contextlength,0.0)
contextvec=ca(batch)
print("context_vecs.shape:",contextvec.shape)

context_vecs.shape: torch.Size([2, 6, 2])


In [34]:
class MultiHeadAttention(nn.Module):

    def __init__(self,din,dout,contextlength,dropout,numheads,qkvbias=False):
        super().__init__()
        self.heads=nn.ModuleList([CausalAttention(din,dout,contextlength,dropout,qkvbias)
                                  for _ in range(numheads)]
                                )
       
    def forward(self,x):
        return torch.cat([head(x) for head in self.heads], dim =-1) 

In [35]:
torch.manual_seed(123)
contextlength=batch.shape[1]
din,dout=3,2
mha=MultiHeadAttention(din,dout,contextlength,0.0,numheads=2)
contextvec=mha(batch)
print(contextvec)
print("context vectors shape",contextvec.shape)

tensor([[[-0.4519,  0.2216,  0.4772,  0.1063],
         [-0.5874,  0.0058,  0.5891,  0.3257],
         [-0.6300, -0.0632,  0.6202,  0.3860],
         [-0.5675, -0.0843,  0.5478,  0.3589],
         [-0.5526, -0.0981,  0.5321,  0.3428],
         [-0.5299, -0.1081,  0.5077,  0.3493]],

        [[-0.4519,  0.2216,  0.4772,  0.1063],
         [-0.5874,  0.0058,  0.5891,  0.3257],
         [-0.6300, -0.0632,  0.6202,  0.3860],
         [-0.5675, -0.0843,  0.5478,  0.3589],
         [-0.5526, -0.0981,  0.5321,  0.3428],
         [-0.5299, -0.1081,  0.5077,  0.3493]]], grad_fn=<CatBackward0>)
context vectors shape torch.Size([2, 6, 4])


In [ ]:
import os
import urllib.request
